###### Checklist for logs: 

|name (CHANGE TO ID!)| reason acc. to elasticsearch | reason acc to script| |
|----------|----------|----------|-----|
|9ef7ce17ae108085e8a26e1f2046e1cf73e8c976.json|Infrastructure - Matlab License |Infrastructure - Matlab License |y|
|882dafcc748b4f0695ce486241a05ad2.json|?| Test(s) failed|y|
|e850514f59cb3334754f2641fc63f20f46af56e7.json|FuSa Violation found|FuSa Violation found + Fusa category small checks|?|
|0752ac127d21a70af1765ae29afc53d978224f25.json|python|python|y|
|8e89be688f5b331654a6f44975a7c6c23dd3b71c.json|Compiler (gnu)  |Bazel - error executing command (ERROR:.{1,500}error executing command \(.*\))|X|
|00e0b3d030f7fd0eef0c4b016e3d5e7e2b51a69f.json|Bazel (failed on target analysis)|Not flagged / No match found|X|
|b819bae8dfb1923b8f9dc3c1b4bb2eed8c3fa2a3.json|Compiler (clang)|Bazel - error executing command|X|
|cd5c0bbd5821d9d31f41442f78bcae86221e65a0.json|Bazel(missing input file)|Bazel (missing input file) |?|

In [1]:
## **What is the difference(in error_patterns.py) b/w those that work / don't work? **

# - Python (correct): 
BUILD_PATTERNS = {
    "python": {
        "Authentication error": [r"^.*Traceback.+\n( .+\n)+.*pip/download.py.+handle_401.*$"],
        "guestfs_launch failed": [r"RuntimeError: guestfs_launch failed"],
        "HTTPError": [r"^requests.exceptions.HTTPError: .+$"],
        "pip": [r"ERROR: Pip failed:.*"],
        "pip install failed": [
            r"^ERROR: No matching distribution found for.*$",
            r"^Could not install packages due to an.*$",
        ],
        "Failed to install python dependencies": [
            r"^ERROR: could not install deps .+$",
            r"Preparing metadata \(setup\.py\): finished with status 'error'",
        ],
        "libguestfs": [r"RuntimeError: guestfs_launch failed.+$"],
        "pytest": [
            r"(?s)=========================== short test summary info ============================.*?\sin\s.*?=+"
        ],
        "thread": [r"^.*python.+\n( .+\n).*error: can't start new thread"],
        "": [
            r"^(.*Traceback.+\n( .+\n)+(?P<type>\S.+Error.*?):.+\n?(?P<details>(\s.+\n)*)?)+",
            r"ERROR: .*: This target is being built for Python 3 but \(transitively\) includes Python 2-only sources.",
            r"\/usr\/bin\/python.*: No module named .*$",
        ],
        "unrecognized arguments": [r"(?P<file>[\w-]+\.py): error: unrecognized arguments:.+$"],
        "Exception raised": [
            r"Exception:(.*)Check missing paths here(.*)",
        ],
    }
}

# - Compiler (gnu / clang ...) (not correct):
BUILD_PATTERNS = {
    "Compiler": {
        "clang-tidy": [
            r"".join(
                [
                    r"(?s)^ERROR: .*Running clang-tidy on file (?P<file>.*) failed: ",
                    r"\(Exit.*(?P<details>INFO: \d+ counting clang-tidy .*\nINFO:.*\n.*?",
                    r"At least one clang-tidy finding was treated as error)",
                ]
            )
        ],
        "clang": [r"^ERROR: .*: Compiling .* failed: \(Exit \d\): clang\+\+ failed: error executing command"],
        "gnu": [
            r"^ERROR: .*: Compiling .* failed: \(Exit \d\): x86_64-linux-gnu-g\+\+ failed: error executing command.*$"
        ],
    }
}

# - FuSa (correct but where does additional info stem from?):
BUILD_PATTERNS = {
    "FuSa Violation Found (Check 'bazel_wrapper_log.txt' on the logs folder)": [
        r"Failed actions detected in bazel_wrapper_log.txt file"
    ],
    "FuSa Violation Found (Check 'mitguard_report.html' on the logs folder)": [r"Mitguard check failure"],
    "FuSa Violation Found (Check the log on 'Nightwatch' folder)": [r"Nightwatch verdict: rejected\."]
}